In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import pandas as pd
import pyarrow.parquet as pq

In [2]:
client = Client(n_workers=1)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 12,Total memory: 31.88 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:52389,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 31.88 GiB
Comm: tcp://127.0.0.1:52396,Total threads: 12
Dashboard: http://127.0.0.1:52397/status,Memory: 31.88 GiB
Nanny: tcp://127.0.0.1:52392,


In [10]:
#df = dd.read_parquet('data/train_data_pqt.parquet', columns=['user_id','request_cnt','url_host'])
df = pq.read_table('data/train_data_pqt.parquet',columns=['user_id','request_cnt','url_host'])

In [5]:
df.compute().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210730009 entries, 0 to 210730008
Data columns (total 3 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      int32 
 1   request_cnt  int8  
 2   url_host     object
dtypes: int32(1), int8(1), object(1)
memory usage: 2.6+ GB


In [12]:
data_target_file = pd.read_parquet('data/public_train.pqt')

data_agg = df.select(['user_id', 'url_host', 'request_cnt'])
data_id_url = data_agg.to_pandas()

data_id_url_sum_male = data_id_url.merge(data_target_file[['is_male','user_id']], on = 'user_id', how = 'inner')
data_id_url_sum_male = data_id_url_sum_male.loc[~(data_id_url_sum_male['is_male'].isna()) & (data_id_url_sum_male['is_male'] != 'NA')]
data_id_url_sum_male['is_male'] = data_id_url_sum_male['is_male'].astype('int8')

data_id_url_sum_male['male_request_cnt'] = data_id_url_sum_male['is_male'] * data_id_url_sum_male['request_cnt']
data_id_url_sum_male = data_id_url_sum_male.groupby('url_host').agg({'request_cnt':'sum','male_request_cnt':'sum'})
data_id_url_sum_male['male_fraction'] = data_id_url_sum_male['male_request_cnt']/data_id_url_sum_male['request_cnt']
display(data_id_url_sum_male.sort_values('request_cnt').tail())
display(data_id_url_sum_male.info())

,request_cnt,male_request_cnt,male_fraction
url_host,,,
avatars.mds.yandex.net,19104346,10637927,0.556833
vk.com,22264707,11310971,0.508022
i.ytimg.com,22636425,12727769,0.562269
yandex.ru,30558229,16038656,0.524856
googleads.g.doubleclick.net,31510446,16492100,0.523385


<class 'pandas.core.frame.DataFrame'>
Index: 170833 entries, -1 to яркнига24.рф
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   request_cnt       170833 non-null  int64  
 1   male_request_cnt  170833 non-null  int64  
 2   male_fraction     170833 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 5.2+ MB


None